In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
data = pd.read_csv("stroke.csv", index_col=0)

In [3]:
data.head()

,gender,age,hypertension,alcohol_intake,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,,
9046,Male,67.0,147,2,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,135,4,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,115,3,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,125,2,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,130,3,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
data['hypertension'] = data['hypertension'].astype(float) 
data['alcohol_intake'] = data['alcohol_intake'].astype(float) 

In [5]:
data = data.drop("ever_married" , axis=1)
data = data.drop("Residence_type" , axis=1)

In [6]:
for column in data.columns:
    if data[column].dtype == np.number:
        continue
    data[column] = LabelEncoder().fit_transform( data[column] )

In [7]:
data = data.fillna(data.mean())

In [8]:
X = data.drop(["stroke"], axis=1)
Y = data["stroke"]

In [9]:
x_scaler = MinMaxScaler()
x_scaler.fit(X)
column_names = X.columns
X[column_names]=x_scaler.transform(X)

In [10]:
X.tail()

,gender,age,hypertension,alcohol_intake,heart_disease,work_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,
4750,1.0,0.951172,0.865385,0.000000,0.0,0.50,0.137678,0.180985,0.333333
65127,0.0,0.426270,0.762821,0.666667,0.0,0.50,0.117957,0.211913,1.000000
4498,1.0,0.865723,0.884615,0.833333,1.0,0.50,0.691668,0.215380,0.333333
32203,0.0,0.694824,0.814103,0.333333,0.0,0.75,0.185387,0.253150,0.333333
27436,1.0,0.145508,0.538462,0.000000,0.0,1.00,0.254526,0.115693,0.000000


In [13]:
data.tail(16)

,gender,age,hypertension,alcohol_intake,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
36488,1,12.0,119.0,0.0,0,4,111.47,32.300000,2,0
55567,0,76.0,151.0,2.0,1,2,86.09,28.100000,2,0
33562,1,71.0,152.0,1.0,1,0,72.94,32.300000,1,0
20006,0,15.0,92.0,0.0,0,2,76.77,21.700000,0,0
47696,1,44.0,115.0,1.0,0,2,60.32,25.000000,2,0
60117,1,30.0,101.0,2.0,0,2,133.24,28.900000,2,0
5032,0,47.0,110.0,2.0,0,2,65.01,21.700000,1,0
5780,0,47.0,114.0,2.0,0,2,74.63,45.300000,2,0
52236,0,60.0,145.0,4.0,0,2,230.78,40.200000,2,0


In [15]:
data['work_type'].value_counts()
# Never worked-1,Govt_job-0,Private-2,Self-employed-3,childern-4

2    1767
3     478
4     377
0     365
1      13
Name: work_type, dtype: int64

In [16]:
X=np.array(X)
Y= np.array(Y)

In [17]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_res, Y_res = sm.fit_sample(X,Y.ravel())

In [18]:
print("After OverSampling, counts of label '1': {}".format(sum(Y_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(Y_res == 0))) 

After OverSampling, counts of label '1': 2751
After OverSampling, counts of label '0': 2751


In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.metrics import binary_crossentropy

In [20]:
model = Sequential([
    Dense(32, input_shape=(X_res.shape[1],), activation='relu'),
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax'),    
])

In [21]:
model.compile(optimizer=Adam(learning_rate = 0.0001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [22]:
model.fit(X_res,Y_res,validation_split=0.1,batch_size=10,epochs=140,shuffle=True,verbose=2)

Train on 4951 samples, validate on 551 samples
Epoch 1/140
4951/4951 - 3s - loss: 0.6596 - accuracy: 0.6205 - val_loss: 0.7606 - val_accuracy: 0.3485
Epoch 2/140
4951/4951 - 1s - loss: 0.6015 - accuracy: 0.6890 - val_loss: 0.6988 - val_accuracy: 0.6080
Epoch 3/140
4951/4951 - 2s - loss: 0.5393 - accuracy: 0.7485 - val_loss: 0.5496 - val_accuracy: 0.8040
Epoch 4/140
4951/4951 - 1s - loss: 0.5010 - accuracy: 0.7708 - val_loss: 0.5523 - val_accuracy: 0.7877
Epoch 5/140
4951/4951 - 1s - loss: 0.4799 - accuracy: 0.7736 - val_loss: 0.5539 - val_accuracy: 0.7840
Epoch 6/140
4951/4951 - 2s - loss: 0.4678 - accuracy: 0.7778 - val_loss: 0.5722 - val_accuracy: 0.7768
Epoch 7/140
4951/4951 - 1s - loss: 0.4606 - accuracy: 0.7839 - val_loss: 0.4707 - val_accuracy: 0.8203
Epoch 8/140
4951/4951 - 1s - loss: 0.4561 - accuracy: 0.7867 - val_loss: 0.5043 - val_accuracy: 0.8058
Epoch 9/140
4951/4951 - 1s - loss: 0.4532 - accuracy: 0.7873 - val_loss: 0.5112 - val_accuracy: 0.8040
Epoch 10/140
4951/4951 - 2

Epoch 80/140
4951/4951 - 1s - loss: 0.3862 - accuracy: 0.8311 - val_loss: 0.4247 - val_accuracy: 0.8348
Epoch 81/140
4951/4951 - 1s - loss: 0.3855 - accuracy: 0.8271 - val_loss: 0.4382 - val_accuracy: 0.8294
Epoch 82/140
4951/4951 - 1s - loss: 0.3844 - accuracy: 0.8293 - val_loss: 0.4184 - val_accuracy: 0.8403
Epoch 83/140
4951/4951 - 2s - loss: 0.3849 - accuracy: 0.8285 - val_loss: 0.4888 - val_accuracy: 0.8094
Epoch 84/140
4951/4951 - 1s - loss: 0.3843 - accuracy: 0.8299 - val_loss: 0.4660 - val_accuracy: 0.8167
Epoch 85/140
4951/4951 - 1s - loss: 0.3827 - accuracy: 0.8311 - val_loss: 0.4317 - val_accuracy: 0.8312
Epoch 86/140
4951/4951 - 1s - loss: 0.3821 - accuracy: 0.8295 - val_loss: 0.4326 - val_accuracy: 0.8312
Epoch 87/140
4951/4951 - 2s - loss: 0.3818 - accuracy: 0.8322 - val_loss: 0.4135 - val_accuracy: 0.8385
Epoch 88/140
4951/4951 - 1s - loss: 0.3802 - accuracy: 0.8330 - val_loss: 0.4449 - val_accuracy: 0.8294
Epoch 89/140
4951/4951 - 1s - loss: 0.3801 - accuracy: 0.8354 - 

In [23]:
user_input=input("Enter the values one by one")
user_input=user_input.split(",")


for i in range(len(user_input)):
     #convert each item to int type
    user_input[i] = float(user_input[i])
    

Enter the values one by one1,12,119,0,0,4,111.47,32.22,2


In [24]:
user_input = np.array(user_input)
user_input = user_input.reshape(1,-1)
user_input = x_scaler.transform(user_input)
pred = model.predict(user_input)
print(np.argmax(pred[0]))

0


In [25]:
model.save("finalmodel.h5")

In [26]:
import joblib

In [27]:
joblib.dump(x_scaler,'scaler.pkl')

['scaler.pkl']